In [1]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Flatten

In [2]:
df = pd.read_csv('imdb_master.csv',encoding='latin-1')
print(df.head())
df = df[df['label']!='unsup'] #Dropping Unnecessary labelfrom dataset
sentences = df['review'].values
y = df['label'].values

   Unnamed: 0  type                                             review label  \
0           0  test  Once again Mr. Costner has dragged out a movie...   neg   
1           1  test  This is an example of why the majority of acti...   neg   
2           2  test  First of all I hate those moronic rappers, who...   neg   
3           3  test  Not even the Beatles could write songs everyon...   neg   
4           4  test  Brass pictures (movies is not a fitting word f...   neg   

          file  
0      0_2.txt  
1  10000_4.txt  
2  10001_1.txt  
3  10002_3.txt  
4  10003_3.txt  


In [3]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

Tokenizing data


In [4]:
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(sentences)
max_review_len = max([len(s.split()) for s in sentences])
vocab_size = len(tokenizer.word_index)+1
X_train, X_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)
padded_train = pad_sequences(X_train_tokens,maxlen=max_review_len)
paded_test = pad_sequences(X_test_tokens,maxlen=max_review_len)

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_review_len))
model.add(Flatten())
model.add(layers.Dense(300, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid')) #changing number of neuron to 2 as we have only two labels Pos and Neg
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
history=model.fit(padded_train,y_train, epochs=5, verbose=True, validation_data=(paded_test,y_test), batch_size=256)

Epoch 1/5
147/147 [==============================] - 246s 2s/step - loss: 0.6245 - acc: 0.7173 - val_loss: 0.2998 - val_acc: 0.8761
Epoch 2/5
147/147 [==============================] - 221s 2s/step - loss: 0.2534 - acc: 0.8978 - val_loss: 0.2800 - val_acc: 0.8814
Epoch 3/5
147/147 [==============================] - 265s 2s/step - loss: 0.1858 - acc: 0.9314 - val_loss: 0.3062 - val_acc: 0.8729
Epoch 4/5
147/147 [==============================] - 269s 2s/step - loss: 0.1170 - acc: 0.9646 - val_loss: 0.3243 - val_acc: 0.8736
Epoch 5/5
147/147 [==============================] - 267s 2s/step - loss: 0.0586 - acc: 0.9888 - val_loss: 0.3750 - val_acc: 0.8678


In [6]:
test_loss, test_acc = model.evaluate(paded_test, y_test)
print(test_acc)

391/391 [==============================] - 29s 75ms/step - loss: 0.3750 - acc: 0.8678
0.8678399920463562
